In [1]:
import guardrails as gd
import openai
from rich import print as rprint
import tiktoken
import dotenv
import os
import pandas as pd

from app.database import Database, db_config

from app.guardrails_specs.specs import rail_str

dotenv.load_dotenv()
OPEN_API_KEY = os.getenv("OPENAI_API_KEY")
openai.api_key = OPEN_API_KEY


## Testing execution times

In [2]:
from app.transaction_parser import TransactionParser

test_str = '''Jul 22, 2023	AMZ*Sleepsheep Direct		$141.75	$4,838.90
Jul 22, 2023	FIVE GUYS BURGERS & FR	$71.12		$4,980.65
Jul 22, 2023	ABC*ANYTIME FITNESS	$28.34		$4,909.53
Jul 21, 2023	RING MULTICAM MTH PLAN	$15.75		$4,881.19
Jul 19, 2023	DOORDASH*LUNA MEXICAN	$130.82		$4,865.44
Jul 15, 2023	AMZN Mktp CA*F307K57V3	$39.86		$4,315.77
Jul 14, 2023	AMZN Mktp CA*WV57D4SK3	$33.59		$4,275.91
Jul 13, 2023	adidas 2021 Edmonton	$90.32		$3,242.59
Jul 12, 2023	NIKE CANADA - EDMONTON	$580.71		$3,152.27
Jul 12, 2023	GAP OUTLET CA 8893	$172.73		$2,571.56
Jul 10, 2023	APPLE STORE #R411	$10.00		$2,398.83'''

tp = TransactionParser(transaction_string=test_str)


### Parsing transactions

In [3]:
%%time

parsed_transactions = tp.parse_transactions()

1 [('Jul 22, 2023\tAMZ*Sleepsheep Direct\t\t$141.75\t$4,838.90\nJul 22, 2023\tFIVE GUYS BURGERS & FR\t$71.12\t\t$4,980.65\nJul 22, 2023\tABC*ANYTIME FITNESS\t$28.34\t\t$4,909.53\nJul 21, 2023\tRING MULTICAM MTH PLAN\t$15.75\t\t$4,881.19\nJul 19, 2023\tDOORDASH*LUNA MEXICAN\t$130.82\t\t$4,865.44\nJul 15, 2023\tAMZN Mktp CA*F307K57V3\t$39.86\t\t$4,315.77\nJul 14, 2023\tAMZN Mktp CA*WV57D4SK3\t$33.59\t\t$4,275.91\nJul 13, 2023\tadidas 2021 Edmonton\t$90.32\t\t$3,242.59\nJul 12, 2023\tNIKE CANADA - EDMONTON\t$580.71\t\t$3,152.27\nJul 12, 2023\tGAP OUTLET CA 8893\t$172.73\t\t$2,571.56\nJul 10, 2023\tAPPLE STORE #R411\t$10.00\t\t$2,398.83', 1458)]


/Users/fernandomeira/miniforge3/envs/llm_fin/lib/python3.11/site-packages/eliot/json.py:22: FutureWarning: In the future `np.bool` will be defined as the corresponding NumPy scalar.
  if isinstance(o, (numpy.bool, numpy.bool_)):


CPU times: user 174 ms, sys: 20.9 ms, total: 195 ms
Wall time: 37.9 s


In [ ]:
#testing the split function

split_input = tp.token_validator.process(input=tp.transaction_string)

print(len(split_input))

split_input

In [ ]:
#testing the split function

split_input = tp.token_validator.process(input=tp.transaction_string)

print(len(split_input))

split_input

In [ ]:
tp.token_validator.get_token_count(input=tp.transaction_string)

In [ ]:
tp.token_validator.base_prompt_tokens

In [ ]:
import guardrails as gd
from guardrails_specs.specs import rail_str

guard = gd.Guard.from_rail_string(rail_str)

new_llm_call_all = tp.get_guard_chat_completion(input=tp.transaction_string, guard=guard, token_number=2000)

rprint(tp.last_raw_response_from_guard_completion)

In [ ]:
tp.token_validator.get_token_count(input=tp.last_raw_response_from_guard_completion)

In [ ]:
tp.get_chat_completion(prompt=guard.prompt.format(**{'transaction_string': test_str}).source)
rprint(tp.last_response_chat_completion)

### Generating transactions categories

In [ ]:
%%time
cats = tp.generate_transaction_categories()


### Clustering transactions names

In [ ]:
%%time
new_trans = tp.get_transaction_names(new_transactions=pd.DataFrame(parsed_transactions), older_transactions=pd.DataFrame())